# **DATASET KURS USD/IDR - GOOGLE COLAB CODE**

# PERSIAPAN DATASET


## Import Package

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Load Data

In [ ]:
data_bsde = pd.read_csv('/content/data_bsde_per_bulan.csv')
data_usdidr = pd.read_csv('/content/data_usdidr_per_bulan.csv')
data_birate = pd.read_csv('/content/BI_Rate.csv')
data_inflasi = pd.read_csv('/content/Inflasi_convert.csv')

In [ ]:
data_bsde.head()

In [ ]:
data_usdidr.head()

In [ ]:
data_birate.head()

In [ ]:
data_inflasi.head()

## Data Merging of BSDE, USD/IDR, BI Rate, and Inflation Rate

In [ ]:
data_bsde['Date'] = pd.to_datetime(data_bsde['Date'], format='%b %Y')
data_usdidr['Date'] = pd.to_datetime(data_usdidr['Date'], format='%b %Y')
data_birate['Date'] = pd.to_datetime(data_birate['Date'], format='%b %Y')
data_inflasi['Date'] = pd.to_datetime(data_inflasi['Date'], format='%b %Y')

merged_data = pd.merge(data_bsde, data_usdidr, on='Date', suffixes=('_bsde', '_usdidr'))
merged_data = pd.merge(merged_data, data_birate, on='Date')
merged_data_inflasi = pd.merge(data_bsde, data_inflasi, on='Date')

## Data Visualization of BSDE, USD/IDR, BI Rate, and Inflation Rate


In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.set_xlabel('Date')
ax1.set_ylabel('BSDE Close Price (IDR)', color='blue')
ax1.plot(merged_data['Date'], merged_data['Close_bsde'], label='BSDE Close Price', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.set_ylabel('USD/IDR', color='red')
ax2.plot(merged_data['Date'], merged_data['Close_usdidr'], label='USD/IDR', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('BSDE vs USD/IDR', fontsize=14)

fig.autofmt_xdate()

plt.grid(True)
plt.tight_layout()
plt.show()

fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.set_xlabel('Date')
ax1.set_ylabel('BSDE Close Price (IDR)', color='blue')
ax1.plot(merged_data['Date'], merged_data['Close_bsde'], label='BSDE Close Price', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.set_ylabel('BI Rate (%)', color='red')
ax2.plot(merged_data['Date'], merged_data['BI Rate'], label='BI Rate', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('BSDE vs BI Rate', fontsize=14)

fig.autofmt_xdate()

plt.grid(True)
plt.tight_layout()
plt.show()

fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.set_xlabel('Date')
ax1.set_ylabel('BSDE Close Price (IDR)', color='blue')
ax1.plot(merged_data_inflasi['Date'], merged_data_inflasi['Close'], label='BSDE Close Price', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Inflation Rate (%)', color='red')
ax2.plot(merged_data_inflasi['Date'], merged_data_inflasi['Inflasi'], label='Inflation Rate', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('BSDE vs Inflation Rate', fontsize=14)

fig.autofmt_xdate()

plt.grid(True)
plt.tight_layout()
plt.show()

## Data Preprocessing

### Merging Datasets for BSDE, BI Rate, and USD/IDR Data

This cell merges data for BSDE’s stock price, BI Rate, and USD/IDR exchange rate into a single DataFrame for analysis and visualization.

In [ ]:
merged_data = pd.merge(data_bsde, data_birate, on='Date', how='inner')

merged_data = pd.merge(merged_data, data_usdidr, on='Date', how='inner')

merged_data.rename(columns={
    'Close_x': 'Close BSDE',
    'Close_y': 'Close USDIDR'
}, inplace=True)
merged_data['Date'] = pd.to_datetime(merged_data['Date']).dt.strftime('%b %Y')

In [ ]:
merged_data.head()

### Merging Inflation Rate Data with BSDE, BI Rate, and USD/IDR Dataset

This cell integrates the inflation rate data with the previously merged dataset, which includes BSDE’s close price, BI Rate, and USD/IDR exchange rate.

In [ ]:
merged_data['Date'] = pd.to_datetime(merged_data['Date'], format='%b %Y')
data_inflasi['Date'] = pd.to_datetime(data_inflasi['Date'], format='%b %Y')

merged_data = pd.merge(merged_data, data_inflasi, on='Date', how='inner')

merged_data.rename(columns={
    'Inflasi': 'Inflasi Rate'
}, inplace=True)
merged_data['Date'] = pd.to_datetime(merged_data['Date']).dt.strftime('%b %Y')

In [ ]:
merged_data.head()

## Data Overview and Statistical Summary

In [ ]:
merged_data.info()

In [ ]:
merged_data.describe()

# MODEL 1 SAHAM


## Import Package

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

## Data preparation

### Defining Train and Label Variables for Modeling

In [ ]:
X = merged_data[['Close USDIDR', 'BI Rate', 'Inflasi Rate']].values  # Variabel independen
Y = merged_data['Close BSDE'].values  # Variabel dependen

### Data Splitting and Normalization for Training and Validation

In [ ]:
# 70% Train, 30% Val
total_size = len(X)
train_size = int(0.70 * total_size)

X_train, Y_train = X[:train_size], Y[:train_size]
X_val, Y_val = X[train_size:], Y[train_size:]

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)

scaler_Y = StandardScaler()
Y_train_scaled = scaler_Y.fit_transform(Y_train.reshape(-1, 1)).flatten()
Y_val_scaled = scaler_Y.transform(Y_val.reshape(-1, 1)).flatten()


## Modelling

### Model Architecture


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='linear', kernel_regularizer=regularizers.l2(0.0001))
])

In [ ]:
model.summary()

### Define Checkpoint

In [ ]:
checkpoint_filepath = 'best_model.keras'
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min')

### Model Training

Model Compilation: Setting Optimizer, Loss Function, and Evaluation Metrics

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

Model Training: Fitting the Neural Network on Training Data


In [ ]:
history = model.fit(X_train_scaled, Y_train_scaled,
                    validation_data=(X_val_scaled, Y_val_scaled),
                    epochs=1000,
                    batch_size=32,
                    callbacks=[checkpoint])

### Visualization of Model Training Metrics

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
mse = history.history['mse']
val_mse = history.history['val_mse']
mae = history.history['mae']
val_mae = history.history['val_mae']
epochs_range = range(1, len(loss) + 1)

fig, axes = plt.subplots(1, 3, figsize=(20, 6))  # Adjust figure size for larger plots
fig.suptitle('BSDE Stock Model Metrics', fontsize=16)


# Plot 1: Training and Validation Loss
axes[0].plot(epochs_range, loss, label='Training Loss', color='blue')
axes[0].plot(epochs_range, val_loss, label='Validation Loss', color='red')
axes[0].set_title('Training and Validation Loss')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True)

# Plot 2: Training and Validation MSE
axes[1].plot(epochs_range, mse, label='Training MSE', color='green')
axes[1].plot(epochs_range, val_mse, label='Validation MSE', color='orange')
axes[1].set_title('Training and Validation MSE')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('MSE')
axes[1].legend()
axes[1].grid(True)

# Plot 3: Training and Validation MAE
axes[2].plot(epochs_range, mae, label='Training MAE', color='purple')
axes[2].plot(epochs_range, val_mae, label='Validation MAE', color='brown')
axes[2].set_title('Training and Validation MAE')
axes[2].set_xlabel('Epochs')
axes[2].set_ylabel('MAE')
axes[2].legend()
axes[2].grid(True)


plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to prevent overlap

plt.savefig('BSDE.png') # save plot
plt.show()

# Save Model Plot Matrict

In [ ]:
from google.colab import files
files.download('BSDE.png')